# Machine Learning Trading Bot

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

# Import CSV file and create pandas DataFrame

In [ ]:
# Import stocks dataset
stocks_df = pd.read_csv(
    Path('../Resources/[insert_title].csv'),
    index_col= 'date',
    parse_dates = True,
    infer_datetime_format=True
)

# Review the DataFrame
stocks_df.head()

In [ ]:
#Filter the date index and close columns
signals_df = stocks_df.loc[:, ['close']]

# Use pct_change function to generate returns from close prices
signals_df['Actual Returns'] = signals_df['close'].pct_change()

#Drop all NaN values from the DataFrame
signals_df = signals_df.dropna()

#Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

# Create a trading algorithm using technical indicators

## Generate Signals - Example using SMA with short and long window

In [ ]:
# Create SMA signals DataFrame
sma_signals_df = stocks_df.loc[:, ["close"]].copy()

# Set the short and long windows
short_window = 50
long_window = 100

# Create a short window SMA and long window SMA
sma_signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
sma_signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

sma_signals_df = sma_signals_df.dropna()

# Review the DataFrame
display(sma_signals_df.head())
display(sma_signals_df.tail())

In [ ]:
# Initialize the new Signal column
sma_signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
sma_signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1.0

# When Actual Returns are less than 0, generate signal to sell stock short
sma_signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = 0.0

# Review the DataFrame
display(sma_signals_df.head())
display(sma_signals_df.tail())

In [ ]:
# Visualize close price for the investment
stocks_close = signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["SMA_Fast", "SMA_Slow"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
sma_ema_plot = stocks_close * moving_avgs
sma_ema_plot

## Example using EMA technical indicators

In [ ]:
# Create EMA signals DataFrame
ema_signals_df = stocks_df.loc[:, ["close"]].copy()

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
ema_signals_df["EMA_Short"] = signals_df["close"].ewm(span=short_window).mean()
ema_signals_df["EMA_Long"] = signals_df["close"].ewm(span=long_window).mean()

# Review the DataFrame
ema_signals_df.head()

In [ ]:
# Set the signal column
ema_signals_df['Signal'] = 0

# Generate the trading signal 1 or 0
# Where 1 is when the EMA50 is greater than (i.e. crosses over) the EMA100
# Where 0 is when the EMA 50 is under the EMA100
ema_signals_df['Signal'][short_window:] = np.where(
    ema_signals_df['EMA50'][short_window:] > ema_signals_df['EMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or 0
ema_signals_df['Entry/Exit'] = ema_signals_df['Signal'].diff()

# Review DataFrame
ema_signals_df.head()

In [ ]:
# Review value counts
ema_signals_df['Signal'].value_counts()

In [ ]:
# Visualize entry position relative to close price
entry = ema_signals_df[ema_signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = ema_signals_df[ema_signals_df["Entry/Exit"] == 0.0]["close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = ema_signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = ema_signals_df[["EMA50", "EMA100"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot
